# Using E3 in a Jupyter Notebook - XML

As an API, E3 is designed to be callable from any programming language capable of making API requests. This example will illustrate the process using Python.

**If running locally**, the **requests** library will be installed to access the API. This can be done using the following command in your computer's terminal/command prompt provided you have Python already installed (if **requests** is already installed you can skip this step). A serializer to convert the XML to JSON without having to manually write code to parse the XML file is recommended. We will be using the **xmltodict** library for this purpose. Once the XML is converted to a Python dictionary we can then use Python's built in **JSON** library to prepare the string for the API request. If we want to convert the output to the original XML structure we can install the **dicttoxml** library.

In [ ]:
%pip install -q requests
%pip install -q xmltodict
%pip install -q dicttoxml

**If running Jupyter from the browser** you will need to run the following commands.

In [ ]:
from pyodide.ffi import to_js
from IPython.display import JSON
from js import Object, fetch
%pip install -q xmltodict
%pip install -q dicttoxml

With requests installed the only items remaining are to construct the JSON string to send to the API and to properly construct the API request in python. This notebook will cover constructing the JSON string from scratch. For examples of how to use input from CSV, JSON and XLS/XLSX files see the **E3 Example CSV**, **E3 Example - JSON** and **E3 Example Excel** (in development) files, respectively. 

## Constructing the XML File

The format for the XML file must be correct, otherwise E3 will return an error. In this section we will look at the required format for individual objects within E3 before putting all of them into a single XML file for an API. Once the file is constructed an example of how to use Python to read the XML file and convert it to JSON for the API request will be provided. The discussion here will be focused on formatting, for a more in depth look please see the technical manual for E3. We'll start with the **Analysis** object format. Double click on the cell below to see the XML encoding (Jupyter uses tags for formatting so they are supressed unless the cell is active).

<analysisObject>
    <type>"BCA"</type>
    <projectType>"Infrastructure"</projectType>
    <outputObjects>["required", "measure", "optional"]</outputObjects>
    <studyPeriod>50</studyPeriod>
    <timestepValue>"Year"</timestepValue>
    <timestepComp>"End_Of_Year"</timestepComp>
    <outputReal>true</outputReal>
    <interestRate>0.03</interestRate>
    <discountRateReal>0.03</discountRateReal>
    <inflationRate>0.023</inflationRate>
    <marr>0.03</marr>
    <reinvestRate>0.03</reinvestRate>
    <federalIncomeRate>0.22</federalIncomeRate>
    <otherIncomeRate>0.05</otherIncomeRate>
    <numberOfAlternatives>2</numberOfAlternatives>
    <baseAlternative>0</baseAlternative>
</analysisObject>

There should only be one Analysis object for the input. **Alternative** objects have the following basic format.

<id>0</id>
<name>"Alt 0 Status Quo"</name>
<bcns>[0]</bcns>

Multiple alternatives are required for analysis since E3 assumes comparison to a baseline. If only one alternative exists in an analysis you can simply define an alternative with a single cost object with zero quant or quantValue (See the examples for BCNs. An example with two alternatives is presented here. Note that the **alternativeObjects** key is now prepended and the  associated value is now a list of alternative objects.

<alternativeObjects>[
    <id>0</id>
    <name>"Alt 0 Status Quo"</name>
    <bcns>[0]</bcns>,
    <id>1</id>
    <name>"Alt 1 Build Monument"</name>
    <bcns>[1,2,3,4]</bcns>
]</alternativeObjects>

An individual **BCN** object has the following format if the BCN doesn't recur.

<id>0</id>
<altIds>[0]</altIds>
<type>"Cost"</type>
<subType>"Direct"</subType>
<name>"Investment Cost - Status Quo"</name>
<tags>"Initial Investment"</tags>
<initialOccurrence>0</initialOccurrence>
<real>true</real>
<invest>true</invest>
<residualValue>false</residualValue>
<residualValueOnly>false</residualValueOnly>
<quantityValue>0.0</quantityValue>
<quantity>1.0</quantity>

If the BCN does recur it has the following general format.

<id>2</id>
<altIds>[1]</altIds>
<type>"Cost"</type>
<subType>"Direct"</subType>
<name>"Maintain Monument"</name>
<tags>"OMR Costs"</tags>
<initialOccurrence>1</initialOccurrence>
<real>true</real>
<invest>false</invest>
<life>50</life>
<residualValue>false</residualValue>
<residualValueOnly>false</residualValueOnly>
<recur>
    <interval>1</interval>
    <end>50</end>
    <varRate>"Percent_Delta"</varRate>
    <varValue>0.0</varValue>
</recur>,
<quantityValue>1000.0</quantityValue>
<quantity>1.0</quantity>

Now putting all BCNs together.

<bcnObjects>[
    <id>0</id>
    <altIds>[0]</altIds>
    <type>"Cost"</type>
    <subType>"Direct"</subType>
    <name>"Investment Cost - Status Quo"</name>
    <tags>"Initial Investment"</tags>
    <initialOccurrence>0</initialOccurrence>
    <real>true</real>
    <invest>true</invest>
    <residualValue>false</residualValue>
    <residualValueOnly>false</residualValueOnly>
    <quantityValue>0.0</quantityValue>
    <quantity>1.0</quantity>
,
    <id>1</id>
    <altIds>[1]</altIds>
    <type>"Cost"</type>
    <subType>"Direct"</subType>
    <name>"Construct Monument"</name>
    <tags>"Investment Cost"</tags>
    <initialOccurrence>0</initialOccurrence>
    <real>true</real>
    <invest>true</invest>
    <residualValue>false</residualValue>
    <residualValueOnly>false</residualValueOnly>
    <quantityValue>100000.0</quantityValue>
    <quantity>1.0</quantity>
,
    <id>2</id>
    <altIds>[1]</altIds>
    <type>"Cost"</type>
    <subType>"Direct"</subType>
    <name>"Maintain Monument"</name>
    <tags>"OMR Costs"</tags>
    <initialOccurrence>1</initialOccurrence>
    <real>true</real>
    <invest>false</invest>
    <life>50</life>
    <residualValue>false</residualValue>
    <residualValueOnly>false</residualValueOnly>
    <recur>
        <interval>1</interval>
        <end>50</end>
        <varRate>"Percent_Delta"</varRate>
        <varValue>0.0</varValue>
    </recur>
    <quantityValue>1000.0</quantityValue>
    <quantity>1.0</quantity>
,
    <id>3</id>
    <altIds>[1]</altIds>
    <type>"Benefit"</type>
    <subType>"Direct"</subType>
    <name>"Free Publicity"</name>
    <tags>"Publicity"</tags>
    <initialOccurrence>1</initialOccurrence>
    <real>true</real>
    <invest>false</invest>
    <residualValue>false</residualValue>
    <residualValueOnly>false</residualValueOnly>
    <quantityValue>50000.0</quantityValue>
    <quantity>1.0</quantity>
,
    <id>4</id>
    <altIds>[1]</altIds>
    <type>"Benefit"</type>
    <subType>"Externality"</subType>
    <name>"Increased Tourism"</name>
    <tags>"Tourism"</tags>
    <initialOccurrence>1</initialOccurrence>
    <real>true</real>
    <invest>false</invest>
    <residualValue>false</residualValue>
    <residualValueOnly>false</residualValueOnly>
    <recur>
        <interval>1</interval>
    </recur>
    <quantityValue>5000.0</quantityValue>
    <quantity>1.0</quantity>
]</bcnObjects>

With all the individual pieces in place the final XML string can be constructed. Note that the following will be converted to a python dictionary, hence the capitalized **True** and **False**. This dictionary will then be converted to the JSON string using python's JSON library. If you want to work with JSON directly see the **E3 Example - JSON file**.

<?xml version="1.0" encoding"UTF-8" ?>
<root>
    <analysisObject>
        <type>BCA</type>
        <projectType>Infrastructure</projectType>
        <outputObjects>
            <output0>required</output0>
            <output1>measure</output1> 
            <output2>optional</output2>
        </outputObjects>
        <studyPeriod>50</studyPeriod>
        <timestepValue>Year</timestepValue>
        <timestepComp>End_Of_Year</timestepComp>
        <outputReal>true</outputReal>
        <interestRate>0.03</interestRate>
        <discountRateReal>0.03</discountRateReal>
        <inflationRate>0.023</inflationRate>
        <marr>0.03</marr>
        <reinvestRate>0.03</reinvestRate>
        <federalIncomeRate>0.22</federalIncomeRate>
        <otherIncomeRate>0.05</otherIncomeRate>
        <numberOfAlternatives>2</numberOfAlternatives>
        <baseAlternative>0</baseAlternative>
    </analysisObject>
    <alternativeObjects>
        <alt1>
            <id>0</id>
            <name>Alt 0 Status Quo</name>
            <bcns>
                <bcnItem0>0</bcnItem0>
            </bcns>
        </alt1>
        <alt2>
            <id>1</id>
            <name>Alt 1 Build Monument</name>
            <bcns>
                <bcnItem0>1</bcnItem0> 
                <bcnItem1>2</bcnItem1>
                <bcnItem2>3</bcnItem2>
                <bcnItem3>4</bcnItem3>
            </bcns>
        </alt2>
    </alternativeObjects>
    <bcnObjects>
        <bcn0>
            <id>0</id>
            <altIds>
                <altID0>0</altID0>
            </altIds>
            <type>Cost</type>
            <subType>Direct</subType>
            <name>Investment Cost - Status Quo</name>
            <tags>Initial Investment</tags>
            <initialOccurrence>0</initialOccurrence>
            <real>true</real>
            <invest>true</invest>
            <residualValue>false</residualValue>
            <residualValueOnly>false</residualValueOnly>
            <quantityValue>0.0</quantityValue>
            <quantity>1.0</quantity>
        </bcn0>
        <bcn1>
            <id>1</id>
            <altIds>
                <altID0>1</altID0>
            </altIds>
            <type>Cost</type>
            <subType>Direct</subType>
            <name>Construct Monument</name>
            <tags>Investment Cost</tags>
            <initialOccurrence>0</initialOccurrence>
            <real>true</real>
            <invest>true</invest>
            <residualValue>false</residualValue>
            <residualValueOnly>false</residualValueOnly>
            <quantityValue>100000.0</quantityValue>
            <quantity>1.0</quantity>
        </bcn1>
        <bcn2>
            <id>2</id>
            <altIds>
                <altID0>1</altID0>
            </altIds>
            <type>Cost</type>
            <subType>Direct</subType>
            <name>Maintain Monument</name>
            <tags>OMR Costs</tags>
            <initialOccurrence>1</initialOccurrence>
            <real>true</real>
            <invest>false</invest>
            <life>50</life>
            <residualValue>false</residualValue>
            <residualValueOnly>false</residualValueOnly>
            <recur>
                <interval>1</interval>
                <varRate>Percent_Delta</varRate>
                <varValue>0.0</varValue>
            </recur>
            <quantityValue>1000.0</quantityValue>
            <quantity>1.0</quantity>
        </bcn2>
        <bcn3>
            <id>3</id>
            <altIds>
                <altID0>1</altID0>
            </altIds>
            <type>Benefit</type>
            <subType>Direct</subType>
            <name>Free Publicity</name>
            <tags>Publicity</tags>
            <initialOccurrence>1</initialOccurrence>
            <real>true</real>
            <invest>false</invest>
            <residualValue>false</residualValue>
            <residualValueOnly>false</residualValueOnly>
            <quantityValue>50000.0</quantityValue>
            <quantity>1.0</quantity>
        </bcn3>
        <bcn4>
            <id>4</id>
            <altIds>
                <altID0>1</altID0>
            </altIds>
            <type>Benefit</type>
            <subType>Externality</subType>
            <name>Increased Tourism</name>
            <tags>Tourism</tags>
            <initialOccurrence>1</initialOccurrence>
            <real>true</real>
            <invest>false</invest>
            <residualValue>false</residualValue>
            <residualValueOnly>false</residualValueOnly>
            <recur>
                <interval>1</interval>
            </recur>
            <quantityValue>5000.0</quantityValue>
            <quantity>1.0</quantity>
        </bcn4>
    </bcnObjects>
</root>

XML will generally be stored in a file with the **XML** extention. For the purposes of this example we will use the **input.xml** file (note that the path to the file will change depending on your directory structure). If you examine the file you'll nbotice an additional level to the hierarchy, **root** as the parser we're using for the conversion to JSON requires that the xml structure has a root level.

In [ ]:
import xmltodict

with open('input.xml', 'r') as xf:
    inputDict = xmltodict.parse(xf.read())
    
print(inputDict)

While the data has been brought in, it isn't in a form that E3 will expect. All values are currently treated as strings and certain elements that E3 expects as lists are in a key-value pair format (i.e. the **bcns** attribute is a subdictionary of bcnItemX keys instead of a list of ints). Before converting this to a JSON string we;ll need to clean some of these things up. We'll start with conversions to numeric values where possible. We'll also take this opportunity to convert the boolean expressions to their Python dictionary equivalents.

In [ ]:
def str_to_num(d):
    tol = 1e-6
    for key, value in d.items():
        if isinstance(value, dict):
            str_to_num(value)
        else:
            try:
                valueInt = int(value)
                valueFloat = float(value)
                if abs(float(valueInt)-valueFloat) < tol:
                    d[key] = valueInt
                else:
                    d[key] = valueFloat
            except ValueError:
                try: 
                    d[key] = float(value)
                except ValueError:
                    if value == 'false':
                        d[key] = False
                    elif value == 'true':
                        d[key] = True

str_to_num(inputDict)
print(inputDict)

The numbers have been corrected, however the issue with the lists remains. To correct that we need to first note which keys are expected as lists, for the present example that would be **outputObjects**, **alternativeObjects**, **bcns**, **bcnObjects** and **altIds**. The output has all of these listed as subdictionaries. Fortunately it is a relatively simple matter to convert them to the appropriate form.

In [ ]:
from numpy import ndarray

keys_to_convert = ["outputObjects", "alternativeObjects", "bcns", "bcnObjects", "altIds"]

def subdict_to_list(d, key_list):
    for key, value in d.items():
        if isinstance(value, dict):
            if key in key_list:
                d[key] = list(value.values())
            subdict_to_list(value, key_list)
                
subdict_to_list(inputDict, keys_to_convert)
print(inputDict)

Now all that remains is to prune the corrected dictionary from the **root** key. We do this by noting that the desired dictionary is defined by the key objects **analysisObject**, **alternativeObjects**,and **bcnObjects**. If sensitivity, uncertainty, or any custom modules are being used that alter input they will need to be accounted for in this step.

In [ ]:
key_objects = ["analysisObject", "alternativeObjects", "bcnObjects"]

finalInputDict = dict((object_key, inputDict["root"][object_key]) for object_key in key_objects if object_key in inputDict["root"])
print(finalInputDict)

An API key is required to use E3. Paste yours where it says API_KEY. To get an API key you need to go to https://e3.nist.gov, create a log in and request a new key. The key should be passed in the **Authorization** header. The **Accept** header defines the output the client will accept while the **Content-Type** header defines the format of the sent data.

In [ ]:
headers = {'Content-Type': "application/json", 
           'Accept': "application/json", 
           'Authorization': "Api-Key: CXizFToEOIWa.a8606a6d-d9ae-4907-93a7-ea64c253c37f"
          }

The full request can now be made. **If running locally use the request library and the following code.**

In [ ]:
import requests
import json
api_url = "https://e3.nist.gov/api/v2/analysis"
response = requests.post(api_url, data=json.dumps(finalInputDict), headers=headers)
response.json()
response.status_code

# The status code can be dropped if the request is successful
if response.status_code == 200:
    response = response.json()
    print(response)
else:
    print("Error:", response.status_code)

**If running through JupyterLite use the following code**

In [ ]:
import json

resp = await fetch("https://e3.nist.gov/api/v2/analysis",
                       method = "POST",
                       body = json.dumps(finalInputDict),
                       credentials = "same-origin",
                       headers = Object.fromEntries(to_js(headers))
                      )
response = await resp.text()
response = json.loads(response)

We can now pull output from the reponse object.

In [ ]:
print("Net Benefits: Alt 0 Status Quo:", response['measure'][0]['netBenefits'])
print("Net Benefits: Alt 1 Build Monument:", response['measure'][1]['netBenefits'])
print("BCR: Alt 0 Status Quo:", response['measure'][0]['bcr'])
print("BCR: Alt 1 Build Monument:", response['measure'][1]['bcr'])

Once the request is received it can be converted back into XML format. First we need to convert it to a dictionary using the json library then convert the dictionary to xml using the dicttoxml library.

In [ ]:
import dicttoxml
from xml.dom.minidom import parseString

outputXML = dicttoxml.dicttoxml(response, root = False, attr_type = False)
pretty = parseString(outputXML)
print(pretty.toprettyxml())

From here the xml tags can be further parsed or edited as needed for the analysis or written to a file for display in a browser. Note when writing to bile to drop the characters denoting the output as a string literal (the **b'** and the closing quotation, **'**, at the end of outputXML)